## Pauli Transfer Matrix
The Pauli Transfer Matrix(PTM) of a (potentially unitary) matrix is its representation in Pauli basis, i.e., how it acts on each Pauli string. While this package is based on transforming Pauli strings into one or many Pauli strings, most gates are not defined via the actual PT matrix. However, there are some tools that you can use to work with matrices, both in 0/1 basis and in Pauli basis, potentially to define your own gates. 

In [1]:
using PauliPropagation
using LinearAlgebra

Let us generate a random 1-qubit unitary matrix via Pauli matrix exponentials:

In [2]:
# The Pauli matrices are not exported
using PauliPropagation: Xmat, Ymat, Zmat 

U = exp(-im * (randn() * Xmat + randn() * Ymat + randn() * Zmat))

2×2 Matrix{ComplexF64}:
 0.857247-0.448517im   -0.245791-0.0595636im
 0.245791-0.0595636im   0.857247+0.448517im

Verify that $U$ is unitary via $U \cdot U^\dagger = U^\dagger \cdot U = \mathbb{1}$,

In [3]:
U * U' ≈ U' * U ≈ I(2)

true

This unitary is in the very common 0/1 basis, also called the computational basis.
Here is how you can transform it into the Pauli basis:

In [4]:
# note the default `heisenberg=true` kwarg
Uptm = calculateptm(U)

4×4 Matrix{Float64}:
 1.0   0.0       0.0        0.0
 0.0   0.476839  0.798259  -0.367976
 0.0  -0.739698  0.59057    0.322604
 0.0   0.474837  0.118361   0.872078

This by default returns the PTM of `U` in the **Heisenberg** picture, i.e., how it acts in the backpropagation of Pauli strings - the default in this package.
To get the Schrödinger version, you can take the transpose of this matrix or call `alculateptm(U, heisenberg=false)`.

To convince ourselves that `Uptm` is also a unitary in this basis, check $U_{ptm} \cdot U_{ptm}^T = U_{ptm}^T \cdot U_{ptm} = \mathbb{1}$ due to unitaries being real-valued in this basis.

In [5]:
Uptm * transpose(Uptm) ≈ transpose(Uptm) * Uptm ≈ I(4)

true

Great, but what does this unitary even represent? We mentioned that it represents the action of `U` on Pauli strings. A 1-qubit gate can act on 4 Paulis, `I`, `X`, `Y`, and `Z`, each being represented as $(1, 0, 0, 0)^T$, $(0, 1, 0, 0)^T$, $(0, 0, 1, 0)^T$, and $(0, 0, 0, 1)^T$, respectively. `Uptm` thus describes how each of those column vectors or an arbitrary sum thereof is transformed.

You might already find use for this, but we also support transforming these PTMs into PT maps that can be turned into gates.

In [6]:
ptmap = totransfermap(Uptm)

4-element Vector{Vector{Tuple{UInt8, Float64}}}:
 [(0x00, 1.0)]
 [(0x01, 0.4768394920371281), (0x02, -0.7396984712638481), (0x03, 0.4748370988493844)]
 [(0x01, 0.7982591569056932), (0x02, 0.5905700300300628), (0x03, 0.11836113402001157)]
 [(0x01, -0.3679761096185268), (0x02, 0.322603799172014), (0x03, 0.872078191167396)]

Remember that we encode our Pauli strings in integers, with single-qubit Paulis being 0 (`I`), 1 (`X`), 2 (`Y`), 3 (`Z`). If you index into `ptmap` with those numbers + 1, you will get the corresponding output Paulis with their coefficients. In other words, each entry of a PT map corresponds to a column of the PTM. The Paulis will be set onto the qubit index and the coefficients will be multiplied to the incoming Pauli string's coefficient.

What we also see is that this unitary is 3-branching in Pauli basis. `X`, `Y`, and `Z` Paulis will each map to all three with different coefficients. We can define a `TransferMapGate` from PT map, specifying on what qubit it acts (here qubit 1).

In [7]:
g = TransferMapGate(ptmap, 1)

TransferMapGate{UInt8, Float64}(Vector{Tuple{UInt8, Float64}}[[(0x00, 1.0)], [(0x01, 0.4768394920371281), (0x02, -0.7396984712638481), (0x03, 0.4748370988493844)], [(0x01, 0.7982591569056932), (0x02, 0.5905700300300628), (0x03, 0.11836113402001157)], [(0x01, -0.3679761096185268), (0x02, 0.322603799172014), (0x03, 0.872078191167396)]], [1])

Keep in mind, however, that this gate is not parametrized. It always acts the same.

In [8]:
g isa StaticGate

true

Finally, let us define a circuit consisting of this gate on every qubit: 

In [9]:
# 6 qubits
nq = 6

# define the circuit as a vector of gates
circuit = [TransferMapGate(ptmap, qind) for qind in 1:nq];

# make the observable a rabndom global Pauli string because the gate acts trivially on identities `I`
pstr = PauliString(nq, [rand((:X, :Y, :Z)) for _ in 1:nq], 1:nq)

PauliString(nqubits: 6, 1.0 * ZZZYXX)

In [10]:
psum = propagate(circuit, pstr)

PauliSum(nqubits: 6, 729 Pauli terms:
 0.021565 * ZXYYYX
 0.058295 * XZZYXY
 -0.013901 * YZXYXX
 0.029026 * XYZXYZ
 -0.0074999 * ZZXZZX
 -0.06908 * YZZXYX
 0.015857 * XZXYXX
 -0.006951 * YYXXXX
 -0.050892 * YZZYZY
 -0.0068927 * YYXXZZ
 -0.18674 * ZZZXYX
 0.0058411 * XXYYZX
 -0.0069218 * XYYXXZ
 0.029148 * YZXXXY
 0.032669 * YZZYZZ
 -0.025554 * YZYXYX
 0.021433 * XZXXXX
 0.012085 * ZYYYZZ
 -0.027572 * ZZZZZY
 -0.024598 * ZXXYXY
  ⋮)

And there you go, you can now easily define gates from their matrix representations, both in 0/1 basis or Pauli basis.